In [ ]:
import sys
import os
sys.path.insert(1, os.getcwd())
from datetime import date
from datetime import timedelta

from data_retrieval.google_analytics_api_retrieval import GoogleAnalyticsApiRetrieval, GoogleAuthenticationMethod
from data_loader.local_filesystem_loader import save_list_to_csv

CLIENT_SECRETS_FILE = './credentials/credentials.json'
TOKEN_FILE = './credentials/token.json'
view_id = ""


In [ ]:
def get_data_path(module, year, month, day):
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    data_dir_path = f'./data/{module}/{year}/{month_str}/{day_str}'
    file_name = f'{year}_{month_str}_{day_str}.csv'
    full_path = os.path.join(data_dir_path, file_name)
    return full_path

In [ ]:
start_date = date(2021, 12, 25)
end_date = date(2021, 12, 31)

ga = GoogleAnalyticsApiRetrieval(google_authentication_method = GoogleAuthenticationMethod.OAuth,
                                oauth_credentials_filepath= CLIENT_SECRETS_FILE,
                                oauth_token_filepath=TOKEN_FILE,
                                view_id=view_id)

# modules = ['age', 'gender', 'landing_page']
modules = ['gender']

while start_date < end_date:
    for module in modules:
        # print('getting ', module , 'data for date ', start_date)
        print(f'getting {module} data for date {start_date}')
        data = None
        if module == 'age':
            data = ga.get_sessions_by_age(start_date, start_date)
        elif module == 'gender':
            data = ga.get_sessions_by_gender(start_date, start_date)
        elif module == 'landing_page':
            data = ga.get_sessions_by_landing_page(start_date, start_date)

        print('data is ', data)
        file_path = get_data_path(module, start_date.year, start_date.month, start_date.day)
        print('saving file to ', file_path)
        save_list_to_csv(data, file_path)
        start_date = start_date + timedelta(days=1)